In [56]:
from multisensory_playbook import (
    DetectionTask_versatile,
    LinearClassifier,
    Trials
)
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.optimize import fsolve
import joblib
import sys, os
path = "./data"

train = 1
# "s_range": [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] # Sparse to Dense, sparse is less than 0.2
def calculate_pg(ff, k, N=90, correction=1):
    """
    Use ff to calulate pg, given a k and N
    ff: desired filtered fraction of E 
    pg: probability of E(t)=1 in the base_e (generator)
    k : local on-time duration
    N : number time-steps
    """
    buffer = k
    #pg = (1-fsolve(lambda x: ff-(1-x**k)/(1-x**(N)), 0.9))[0] 
    if correction:
        ff = (1-fsolve(lambda x: ff-(1-x**k)/(1-x**(N+int(buffer)-1*(k-1))), 0.9))[0]

    return ff
    
# Tasks
# Detection task
pe_sparse = 0.04
nb_steps = 200 #500 
nb_trials = 200#000 # Original: 100000
classifier_type = LinearClassifier
time_dep = 1 # 1: there is time dependence

pm=1
pc = 0.45



tasks = [
    
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+1-1, k=1), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=1), # sparse     
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+2-1,k=2), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=2)] # sparse  
"""DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+3-1,k=3), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=3), # sparse
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+4-1,k=4), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=4), # sparse
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+5-1,k=5), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=5), # sparse 
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+6-1,k=6), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=6), # sparse 
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+7-1,k=7), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=7), # sparse
    DetectionTask_versatile(pm=pm, pe=calculate_pg(pe_sparse,  N=nb_steps+8-1,k=8), pc=pc, pn=1 / 3, pi=0.01, time_dep=time_dep, k=8) # sparse
    
]"""

k_list = [1,2]#,3,4,5,6,7,8]

prefix_list = ['LF', 'NLF_1']
windowsize_list = [2,3]

modelpath = './data/'

metrics_dict = {}

In [57]:
# Training all classifiers
if train:
    for a, task in enumerate(tasks):
        train_k = k_list[a]
        print(task)

        # Generate training data 
        full_trials_train = task.generate_trials(nb_trials, nb_steps+train_k-1)
        training_trials = Trials(
            repeats=nb_trials,
            time_steps=nb_steps+train_k-1,
            M=full_trials_train.M,
            A=full_trials_train.A,
            V=full_trials_train.V,
            task=task
        )

        for pairs in [0, 1, 2]:

            if pairs in [0,1]:
                classifier = LinearClassifier(task, pairs=pairs)

                # Train and test the classifier using trials generated using sliding window features
                trained_classifier = classifier.train(training_trials)
                trained_classifiers_dict[(train_k, prefix_list[pairs])] = trained_classifier
                print(f"Trained classifier, k: {train_k},  pair: {pairs}, prefix: {prefix_list[pairs]}")

            if pairs == 2:
                for ws in windowsize_list:
                    classifier = LinearClassifier(task, pairs=pairs, windowsize=ws)
        
                    # Train and test the classifier using trials generated using sliding window features
                    trained_classifier = classifier.train(training_trials)
                    trained_classifiers_dict[(train_k, ws)] = trained_classifier
                    print(f"Trained classifier, k: {train_k},  pair: {pairs}, winsize: {ws}")
            
        print('Training complete')    

        
    
    # Save the dictionary to a file
    """
    full_path = f'/trained_classifiers_{nb_steps}steps.pkl'
    # Check if file exists and is not empty
    if os.path.exists(full_path) and os.path.getsize(full_path) > 0:
      # File exists and is not empty, so load the dictionary
        with open(full_path, 'rb') as file:
          loaded_trained_classifiers_dict = pickle.load(file)
        print("Loaded existing trained classifiers from file.")
        loaded_trained_classifiers_dict.update(trained_classifiers_dict) # Add current trained classifiers
      # Write the updated dictionary back to the file
        with open(full_path, 'wb') as file:
            pickle.dump(loaded_trained_classifiers_dict, file)
        print("Updated and saved trained classifiers to file.")
        
        
    else:
    """
    with open(f'./all_trained_classifiers_{nb_steps}steps.pkl', 'wb') as file:
        pickle.dump(trained_classifiers_dict, file)
    print("Created new file and saved trained classifiers.")    

DetectionTask_versatile(pm=1, pe=0.039989045971137616, pn=0.3333333333333333, pc=0.45, pi=0.01, trans_prob=None, time_dep=1, k=1, random_seed=None, do_return=True)
win size:  None
Trained classifier, k: 1,  pair: 0, prefix: LF
win size:  None
Trained classifier, k: 1,  pair: 1, prefix: NLF_1
win size:  2
Trained classifier, k: 1,  pair: 2, winsize: 2
win size:  3
Trained classifier, k: 1,  pair: 2, winsize: 3
Training complete
DetectionTask_versatile(pm=1, pe=0.019848430653309435, pn=0.3333333333333333, pc=0.45, pi=0.01, trans_prob=None, time_dep=1, k=2, random_seed=None, do_return=True)
win size:  None
Trained classifier, k: 2,  pair: 0, prefix: LF
win size:  None
Trained classifier, k: 2,  pair: 1, prefix: NLF_1
win size:  2
Trained classifier, k: 2,  pair: 2, winsize: 2
win size:  3
Trained classifier, k: 2,  pair: 2, winsize: 3
Training complete
Created new file and saved trained classifiers.


In [71]:
for a, task in enumerate(tasks):
    test_k = k_list[a]
    # Generate test data 
    full_trials_test = task.generate_trials(nb_trials, nb_steps+test_k-1)
    
    testing_trials = Trials(
        repeats=nb_trials,
        time_steps=nb_steps+train_k-1,
        M=full_trials_test.M,
        A=full_trials_test.A,
        V=full_trials_test.V,
        task=task
    )
    
    for train_k in k_list:
        pairs = 0
           
                    
        trained_classifier = trained_classifiers_dict[(train_k, prefix_list[pairs])]
        
        res = trained_classifier.test(testing_trials)
        
        metrics_dict[(train_k, prefix_list[pairs], test_k)] = {
            'accuracy': res.accuracy,
            'precision': res.precision.tolist(),  # Convert numpy array to list for easier storage
            'recall': res.recall.tolist(),
            'f1': res.f1.tolist(),
            'support': res.support.tolist()
        }
        print(f"Tested classifier, test k: {test_k}, train k: {train_k},  pair: {pairs}, prefix: {prefix_list[pairs]}")
            
    with open(f'./{prefix_list[pairs]}_classifier_metrics_500steps.pkl', 'wb') as file:
        pickle.dump(pickle.dump(metrics_dict, file), file)
    
    for train_k in k_list:
        pairs = 1
           
                    
        trained_classifier = trained_classifiers_dict[(train_k, prefix_list[pairs])]
        
        res = trained_classifier.test(testing_trials)
        
        metrics_dict[(train_k, prefix_list[pairs], test_k)] = {
            'accuracy': res.accuracy,
            'precision': res.precision.tolist(),  # Convert numpy array to list for easier storage
            'recall': res.recall.tolist(),
            'f1': res.f1.tolist(),
            'support': res.support.tolist()
        }
        print(f"Tested classifier, test k: {test_k}, train k: {train_k},  pair: {pairs}, prefix: {prefix_list[pairs]}")
            
    with open(f'./{prefix_list[pairs]}_classifier_metrics_500steps.pkl', 'wb') as file:
        pickle.dump(pickle.dump(metrics_dict, file), file)
    
    for train_k in k_list:
            pairs = 2
            
            for ws in windowsize_list:
                
                trained_classifier = trained_classifiers_dict[(train_k, ws)]
                res = trained_classifier.test(testing_trials)
                
                metrics_dict[(train_k, ws, test_k)] = {
                    'accuracy': res.accuracy,
                    'precision': res.precision.tolist(),  # Convert numpy array to list for easier storage
                    'recall': res.recall.tolist(),
                    'f1': res.f1.tolist(),
                    'support': res.support.tolist()
                }
                print(f"Tested classifier, test k: {test_k}, train k: {train_k},  pair: {pairs}, winsize: {ws}")
        
    with open(f'./NLFw_classifier_metrics_500steps.pkl', 'wb') as file:
        pickle.dump(pickle.dump(metrics_dict, file), file)      



win size:  None
Tested classifier, test k: 1, train k: 1,  pair: 0, prefix: LF
win size:  None
Tested classifier, test k: 1, train k: 2,  pair: 0, prefix: LF
win size:  None
Tested classifier, test k: 1, train k: 1,  pair: 1, prefix: NLF_1
win size:  None
Tested classifier, test k: 1, train k: 2,  pair: 1, prefix: NLF_1
win size:  2
Tested classifier, test k: 1, train k: 1,  pair: 2, winsize: 2
win size:  3
Tested classifier, test k: 1, train k: 1,  pair: 2, winsize: 3
win size:  2
Tested classifier, test k: 1, train k: 2,  pair: 2, winsize: 2
win size:  3
Tested classifier, test k: 1, train k: 2,  pair: 2, winsize: 3
win size:  None
Tested classifier, test k: 2, train k: 1,  pair: 0, prefix: LF
win size:  None
Tested classifier, test k: 2, train k: 2,  pair: 0, prefix: LF
win size:  None
Tested classifier, test k: 2, train k: 1,  pair: 1, prefix: NLF_1
win size:  None
Tested classifier, test k: 2, train k: 2,  pair: 1, prefix: NLF_1
win size:  2
Tested classifier, test k: 2, train k: 

In [59]:
NLFw_path = './Plotter/'
import pickle as pkl
class NoFilesFoundError(Exception):
    """Exception raised when no fusion run files are found."""
    pass
def load_fusion_accuracies(base_path=NLFw_path, steps=500, k_range=range(1, 9), include_levy=False, eql_distr=False, load_metrics=False):
    """
    Load NLFw (Non-Linear Fusion with sliding time windows), NLF or LF accuracies from pickle files and summarize the results.

    This function loads accuracy data from a single pickle file, which can be either for regular k-based (i.e., Non-Levy) data,
    Levy-based data, or equal distribution Levy-based data, depending on the parameters. It then summarizes the data
    using the fusion_summary function.

    Args:
        base_path (str): The base path where the files are stored. Default is NLFw_path.
        steps (int): The number of steps used in the fusion training. Default is 500.
        k_range (range): The range of k values to consider. Default is range(1, 9). (Note: This parameter is unused in the function body)
        include_levy (bool): If True, load Levy-based file instead of regular k-based file. Default is False.
        eql_distr (bool): If True and include_levy is True, load equal distribution Levy file. Default is False.

    Returns:
        pandas.Dictionary: A Dictionary containing the summarized accuracy data.

    Raises:
        NoFilesFoundError: If no matching fusion file is found in the specified path.
    """

    all_runs = {}
    
    name = 'metrics' if load_metrics else 'accuracies'
    
    if not include_levy:
        print("Loading non-Levy files")  # Debug print
        # Regular k-based files
        file_path = os.path.join(base_path, f'classifier_{name}_{steps}steps.pkl')
        print(file_path)
    elif eql_distr:
        file_path = os.path.join(base_path, f'classifier_{name}_{steps}steps_levy_eqldistr_k.pkl')
    else:
        file_path = os.path.join(base_path, f'classifier_{name}_{steps}steps_levy.pkl')
        
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            data = pkl.load(file)
            
            for key, value in data.items():
                if key not in all_runs:
                    all_runs[key] = []
                all_runs[key].append(value)
            
    else:
        raise NoFilesFoundError(f"No file found in {base_path} with steps={steps}")     
    print(f"Number of items in all_runs: {len(all_runs)}")  # Debug print
    #df = fusion_summary(all_runs)
    return all_runs

    
def fusion_summary(datadict, accuracy_mult_by_100=True):
    """
    Summarize NLFw (Non-Linear Fusion with sliding time windows), NLF or LF accuracy data.

    This function takes a dictionary of accuracy data, converts it into a pandas DataFrame,
    and calculates accuracies for each combination of train, test, and window size. 

    Args:
        datadict (dict): A dictionary containing accuracy data.
            The keys are tuples of (train, windowsize, test), and the values are lists of accuracies.
        accuracy_mult_by_100 (bool): If True, multiply the mean accuracy by 100. Default is True.
    

    Returns:
        dict: A dictionary where keys are window sizes and values are pandas DataFrames.
              Each DataFrame contains accuracies for the corresponding window size.
    """
       
    # Create DataFrame
    df = pd.DataFrame([(k[0], k[1], k[2], v) for k, v in datadict.items()], 
                      columns=['train', 'windowsize', 'test', 'accuracy'])
    
    # Explode the accuracy list
    df = df.explode('accuracy')
    
    # Get unique window sizes
    windowsizes = df['windowsize'].unique()
    
    # Split the DataFrame based on windowsize
    dfs = {ws: df[df['windowsize'] == ws] for ws in windowsizes}
    
    # Calculate statistics for each split DataFrame
    stats_dfs = {}
    
    for ws, split_df in dfs.items():
        stats_df = split_df.groupby(['train', 'test'])['accuracy'].agg(['mean']).reset_index()
        stats_df.columns = ['train', 'test', 'mean_accuracy']
        if accuracy_mult_by_100:
            stats_df['mean_accuracy'] *= 100
        stats_df['std_accuracy'] = None # Required column for plotting functionality
        stats_dfs[ws] = stats_df
    return stats_dfs

In [74]:
NLFw_nonlevy = load_fusion_accuracies(steps=500, include_levy=0, eql_distr=0, load_metrics=1)

Loading non-Levy files
./Plotter/classifier_metrics_500steps.pkl
Number of items in all_runs: 256


In [75]:
NLFw_nonlevy

{(1,
  'LF',
  1): [{'accuracy': 0.75,
   'precision': [0.7021276595744681, 0.7924528301886793],
   'recall': [0.75, 0.75],
   'f1': [0.7252747252747253, 0.7706422018348624],
   'support': [88, 112]}],
 (1,
  'NLF_1',
  1): [{'accuracy': 0.725,
   'precision': [0.6853932584269663, 0.7567567567567568],
   'recall': [0.6931818181818182, 0.75],
   'f1': [0.6892655367231638, 0.7533632286995515],
   'support': [88, 112]}],
 (1,
  2,
  1): [{'accuracy': 0.67,
   'precision': [0.6170212765957447, 0.7169811320754716],
   'recall': [0.6590909090909091, 0.6785714285714286],
   'f1': [0.6373626373626373, 0.6972477064220184],
   'support': [88, 112]}],
 (1,
  3,
  1): [{'accuracy': 0.64,
   'precision': [0.5754716981132075, 0.7127659574468085],
   'recall': [0.6931818181818182, 0.5982142857142857],
   'f1': [0.6288659793814433, 0.6504854368932039],
   'support': [88, 112]}],
 (2,
  'LF',
  1): [{'accuracy': 0.76,
   'precision': [0.7040816326530612, 0.8137254901960784],
   'recall': [0.78409090909

In [65]:
NLFw_nonlevy

{(1,
  'LF',
  1): [{'accuracy': 0.75,
   'precision': [0.7021276595744681, 0.7924528301886793],
   'recall': [0.75, 0.75],
   'f1': [0.7252747252747253, 0.7706422018348624],
   'support': [88, 112]}],
 (1,
  'NLF_1',
  1): [{'accuracy': 0.725,
   'precision': [0.6853932584269663, 0.7567567567567568],
   'recall': [0.6931818181818182, 0.75],
   'f1': [0.6892655367231638, 0.7533632286995515],
   'support': [88, 112]}],
 (1,
  2,
  1): [{'accuracy': 0.67,
   'precision': [0.6170212765957447, 0.7169811320754716],
   'recall': [0.6590909090909091, 0.6785714285714286],
   'f1': [0.6373626373626373, 0.6972477064220184],
   'support': [88, 112]}],
 (1,
  3,
  1): [{'accuracy': 0.64,
   'precision': [0.5754716981132075, 0.7127659574468085],
   'recall': [0.6931818181818182, 0.5982142857142857],
   'f1': [0.6288659793814433, 0.6504854368932039],
   'support': [88, 112]}],
 (2,
  'LF',
  1): [{'accuracy': 0.76,
   'precision': [0.7040816326530612, 0.8137254901960784],
   'recall': [0.78409090909